# 교차 검증과 그리드 서치
## 검증 세트

- 앞에서는 훈련 세트(80%) : 테스트 세트(20%) 비율로 나눴음.

- 검증세트는 훈련세트에서 추가로 20% 정도를 떼어냄.

- 검증세트는 테스트 세트를 사용하지 않고 모델이 과대적합인지 과소적합인지 파악하기 위해 사용.

- 훈련 세트에서 모델을 훈련 => 검증 세트로 모델을 평가

- 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름

- 그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련

- 마지막에 테스트 세트에서 최종 점수를 평가

In [3]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('./db/wine_csv_data.csv')

In [4]:
# class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
# 훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
# train_input & train_target 을 다시 훈련 세트와 검증세트로 나누기.
# train_input 의 20%를 val_input 으로 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 훈련 세트에 과대적합 되어있음 => 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

## 교차 검증(cross validation)

- 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복

- 그 다음 이 점수를 평균하여 최종 검증 점수를 얻음

- 사이킷런의 cross_validate() 라는 교차 검증 함수를 사용

In [9]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00543571, 0.00518394, 0.00532508, 0.00534296, 0.00515103]), 'score_time': array([0.00044394, 0.00035906, 0.00041699, 0.00043225, 0.00039005]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
# 교차 점증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- cross_validate() 는 훈련 세트를 섞어 폴드를 나누지 않음 

- 앞에서는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 섞을 필요가 없었음.

- 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(Splitter)를 지정해야 함.

In [11]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


- 이제 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봅시다.

- 이 때, 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 됩니다.

## 하이퍼파라미터 튜닝

- 머신러닝 모델이 학습하는 파라미터 : 모델 파라미터

- 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터 : 하이퍼 파라미터

- 하이퍼 파라미터 튜닝하는 작업 : 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련 => 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꾸면서 훈련

- 매개변수를 바꿔가며 최적의 값을 찾는 도구 : 사이킷런의 그리드 서치(grid search)

- GridSearchCV : 하이퍼파라미터 탐색 & 교차 검증을 한번에 수행

In [1]:
# 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [14]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
# 사이킷런의 그리드 서치는 훈련이 끝나면 5*5=25개 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련합니다.
# 이 가장 검증점수가 높은 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있습니다.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있습니다.
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수 : cv_results_ 속성의 'mean_test_score'키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
# 교차 검증 평균 점수 중 가장 높은 값의 인덱스를 best_index 변수에 넣고,
# 그 인덱스 값을 이용해 params 키에 저장된 매개변수 값을 구하자.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#### 과정 정리
1. 먼저 탐색할 매개변수를 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾기. 이 조합은 그리드 서치 객체에 저장됨.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 검증 세트로 나누기 전 훈련 세트를 사용해 최종 모델을 훈련함. 이 모델도 그리드 서치 객체에 저장됨.

In [20]:
# 더 복잡한 매개변수 조합 탐색
# min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량을 지정함
# max_depth 로 트리의 깊이를 제한
# min_samples_split 으로 노드를 나누기 위한 최소 샘플 수도 골라보자

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 1)
         }

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100)})

In [22]:
# 최상의 매개변수 조합을 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 8}


In [23]:
# 최상의 교차 검증 점수도 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8689635004071963


### 랜덤 서치

- 매개변수 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음.

- 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 기간이 오래 걸릴 수 있음.

- 이럴 때 랜덤 서치를 사용하면 좋음.

- 랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달함.

In [24]:
# stats 서브 패키지에 있는 uniform & randint 클래스 모두 주어진 범위에서 고르게 값을 뽑음.
from scipy.stats import uniform, randint
# randint : 정숫값을 뽑음
# unifrom : 실숫값을 뽑음

In [25]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 6, 1, 9, 1, 0, 2, 9, 5, 4])

In [26]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 88, 103,  98,  99,  98, 105, 111,  87, 106, 105]))

In [27]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.19787999, 0.20001035, 0.66012641, 0.26008324, 0.33949245,
       0.33028626, 0.33982294, 0.50858841, 0.28146018, 0.84732077])

In [28]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
         }

In [29]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17fd64b20>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x17fc16110>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17fd64070>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17fd65ff0>},
                   random_state=42)

In [30]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [32]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [33]:
# 연습문제 3번
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random',
                        random_state=42), params, n_iter=100,
                        n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
